In [15]:
import scipy.sparse as sps
from Recommenders.Hybrid.ItemUserHybridKNNRecommender import ItemUserHybridKNNRecommender
from Utils.Evaluator import EvaluatorHoldout
import Utils.Reader as Reader
from Data_manager.split_functions.split_train_validation_random_holdout import\
    split_train_in_two_percentage_global_sample
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Utils.Evaluator import EvaluatorHoldout
import Utils.Reader as Reader

#URM_all=Reader.load_URM(values_to_replace={0.01:0.02})
URM_all=Reader.read_train_csr(values_to_replace={0:0.03})
URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all ,train_percentage=0.80)


from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GeneralizedMergedHybridRecommender import GeneralizedMergedHybridRecommender
import numpy as np

Unique UserID in the URM are 41629
Unique UserID in the URM and ICM are 41629
5826506


In [ ]:
import os
import itertools
best_result=0
best={}
from tqdm.notebook import tqdm
import numpy
alphaL=numpy.arange(0.10, 0.5, 0.1)
betaL=numpy.arange(0.20, 0.5, 0.1)
gammaL=numpy.arange(0.50, 0.8, 0.05)
blockfact=numpy.arange(0.01, 0.05, 0.01)

rp3=RP3betaRecommender(URM_train)
rp3.fit()

ials=IALSRecommender(URM_train)
ials.fit()

itemKNN=ItemKNNCFRecommender(URM_train)
itemKNN.fit()
slim =SLIM_BPR_Cython(URM_train)
slim.fit()
p3=P3alphaRecommender(URM_train)
p3.fit()
for group_id in tqdm(range(20, 22)):
    for x in itertools.product(alphaL,betaL,gammaL,blockfact):
        alpha=x[0]
        beta=x[1]
        gamma=x[2]
        blockfactor=x[3]
        profile_length = np.ediff1d(sps.csr_matrix(URM_all).indptr)
        sorted_users = np.argsort(profile_length)
        block_size = int(len(profile_length)*blockfactor)
        start_pos = group_id*block_size
        end_pos = min((group_id+1)*block_size, len(profile_length))

        users_in_group = sorted_users[start_pos:end_pos]

        users_in_group_p_len = profile_length[users_in_group]
        users_not_in_group_flag = np.isin(sorted_users, users_in_group, invert=True)
        users_not_in_group = sorted_users[users_not_in_group_flag]

        w1=len(users_in_group)/len(profile_length)
        w2=len(users_not_in_group)/len(profile_length)
        from Utils.Evaluator import EvaluatorHoldout
        from Recommenders.GeneralizedMergedHybridRecommender import GeneralizedMergedHybridRecommender
        import numpy as np



        higher_recommender = GeneralizedMergedHybridRecommender(
            URM_train=URM_train,
            recommenders=[
                rp3,
                ials,
                itemKNN

            ],
            verbose=False
        )
        higher_recommender.fit(
            alphas=[
                alpha,beta,
                1 - alpha - beta
            ]
        )



        lower_recommender = GeneralizedMergedHybridRecommender(
            URM_train=URM_train,
            recommenders=[
                p3,
                slim

            ],
            verbose=False
        )
        lower_recommender.fit(
            alphas=[
                gamma,
                1-gamma
            ]
        )
        cutoff=10
        evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[cutoff], ignore_users=users_not_in_group)
        result_df, _ = evaluator_test.evaluateRecommender(higher_recommender)

        evaluator_test2 = EvaluatorHoldout(URM_test, cutoff_list=[cutoff], ignore_users=users_in_group)
        result_df2, _ = evaluator_test2.evaluateRecommender(lower_recommender)
        r2=result_df2["MAP"][10]
        r=result_df["MAP"][10]
        result=r*w1+r2*w2
        if result>best_result:
            best_result=result
            best={
                "alpha:": str(alpha),
                "beta": str(beta),
                "gamma:": str(gamma),
                "groupid:":str(group_id),
                "block_factor":str(blockfactor),
                "MAP":str(best_result)
            }
            parent_dir =  "../"
            write_path = os.path.join(parent_dir, "Testing_Results/grid.csv")
            with open(write_path, "w+") as d:
                d.write(str(best))
                d.write('\n')


In [21]:
alphaL=numpy.arange(0.10, 0.3, 0.1)
betaL=numpy.arange(0.10, 0.3, 0.1)
gammaL=numpy.arange(0.10, 0.3, 0.1)
blockfact=numpy.arange(0.03, 0.05, 0.01)
for group_id in tqdm(range(17, 22)):
    for x in itertools.product(betaL,gammaL,blockfact):
        beta=x[0]
        gamma=x[1]
        blockfactor=x[2]
        profile_length = np.ediff1d(sps.csr_matrix(URM_all).indptr)
        sorted_users = np.argsort(profile_length)
        block_size = int(len(profile_length)*blockfactor)
        start_pos = group_id*block_size
        end_pos = min((group_id+1)*block_size, len(profile_length))

        users_in_group = sorted_users[start_pos:end_pos]

        users_in_group_p_len = profile_length[users_in_group]
        users_not_in_group_flag = np.isin(sorted_users, users_in_group, invert=True)
        users_not_in_group = sorted_users[users_not_in_group_flag]

        w1=len(users_in_group)/len(profile_length)
        w2=len(users_not_in_group)/len(profile_length)
        from Utils.Evaluator import EvaluatorHoldout
        from Recommenders.GeneralizedMergedHybridRecommender import GeneralizedMergedHybridRecommender
        import numpy as np



        cutoff=10
        evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[cutoff], ignore_users=users_not_in_group)
        result_df, _ = evaluator_test.evaluateRecommender(itemKNN)

        evaluator_test2 = EvaluatorHoldout(URM_test, cutoff_list=[cutoff], ignore_users=users_in_group)
        result_df2, _ = evaluator_test2.evaluateRecommender(ials)
        r2=result_df2["MAP"][10]
        r=result_df["MAP"][10]
        result=r*w1+r2*w2
        if result>best_result:
            best_result=result
            best={
                "groupid:":str(group_id),
                "block_factor":str(blockfactor),
                "MAP":str(best_result)
            }
            parent_dir =  "../"
            write_path = os.path.join(parent_dir, "Testing_Results/grid.csv")
            with open(write_path, "w+") as d:
                d.write(str(best))
                d.write('\n')

  0%|          | 0/5 [00:00<?, ?it/s]

EvaluatorHoldout: Ignoring 321 ( 0.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 40381 Users
EvaluatorHoldout: Processed 1248 (100.0%) in 3.88 sec. Users per second: 322
EvaluatorHoldout: Ignoring 321 ( 0.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 1248 Users


/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),
/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:270: RuntimeWarning: invalid value encountered in double_scalars
  ndcg_ = rank_dcg / ideal_dcg


EvaluatorHoldout: Processed 40060 (100.0%) in 42.37 sec. Users per second: 946
EvaluatorHoldout: Ignoring 321 ( 0.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 39964 Users
EvaluatorHoldout: Processed 1665 (100.0%) in 4.26 sec. Users per second: 391
EvaluatorHoldout: Ignoring 321 ( 0.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 1665 Users


/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),
/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:270: RuntimeWarning: invalid value encountered in double_scalars
  ndcg_ = rank_dcg / ideal_dcg


...

/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),
/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:270: RuntimeWarning: invalid value encountered in double_scalars
  ndcg_ = rank_dcg / ideal_dcg


EvaluatorHoldout: Ignoring 321 ( 0.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 1665 Users


/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),
/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:270: RuntimeWarning: invalid value encountered in double_scalars
  ndcg_ = rank_dcg / ideal_dcg


EvaluatorHoldout: Processed 39643 (100.0%) in 38.04 sec. Users per second: 1042
EvaluatorHoldout: Ignoring 321 ( 0.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 41629 Users
EvaluatorHoldout: WARNING: No users had a sufficient number of relevant items


TypeError: unsupported format string passed to MAP.__format__